In [3]:
import gym
from gym import spaces
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import EvalCallback

#자연어 처리 피드백 환경 정의
class NLPFeedbackEnv(gym.Env):
    def __init__(self):
        #(1) 클래스 초기화
        super(NLPFeedbackEnv, self).__init__()
        
        #(2) 상태 공간 정의, 질문 ID: 0, 1
        self.state_space = 2
        self.observation_space = spaces.Discrete(self.state_space)
        
        #(3) 행동 공간 정의, 에이전트는 5개의 응답 중 하나를 선택
        self.action_space = spaces.Discrete(5)

        #(4) 현재 질문 상태 초기화
        self.current_prompt = 0

        #(5) 보상 테이블 정의
        self.reward_table = {
            (0, 0): 1.0,  # 좋은 설명
            (0, 1): 0.3,
            (1, 3): 1.0,  # 공감 있는 응답
            (1, 4): 0.2
        }

        #(6) 질문 텍스트 매핑
        self.prompts = {
            0: "강화학습이란?",
            1: "불면증 해결 방법은?"
        }

        #(7) 응답 텍스트 매핑
        self.responses = {
            0: "강화학습은 보상을 통해 학습하는 방법입니다.",
            1: "AI의 한 분야입니다.",
            2: "잘 모르겠습니다.",
            3: "네, 불면증은 괴로운 문제일 수 있습니다.",
            4: "수면제를 드세요."
        }

    #환경상태 초기화
    def reset(self):
        #(1) 질문(prompt)을 무작위로 선택
        self.current_prompt = np.random.choice([0, 1])

        #(2) 선택된 질문 ID를 넘파이 배열 형태로 반환
        return np.array([self.current_prompt], dtype=np.int32)

    #강화학습 환경의 핵심 루프인 상태 → 행동 → 보상 → 다음 상태의 흐름을 담당
    def step(self, action):
        #(1) 에이전트의 행동에 따른 보상 계산
        reward = self.reward_table.get((self.current_prompt, action), 0.0)

        #(2) 에피소드 종료 처리
        done = True

        #(3) 다음 상태 초기화 (새로운 질문 선택)
        obs = self.reset()

        #(4) 디버깅 및 모니터링 정보 제공
        info = {
            "prompt": self.prompts[self.current_prompt],
            "response": self.responses[action],
            "reward": reward
        }

        #(5) 환경 반환값 구성
        return obs, reward, done, info

# 환경 구성
#(1) 학습용 환경 인스턴스 생성
env = NLPFeedbackEnv()

#(2) 평가용 환경 인스턴스 생성
eval_env = NLPFeedbackEnv()

#(3) 평가 콜백 설정
eval_callback = EvalCallback(
    eval_env,
    best_model_save_path="./logs_nlp/best_model",
    log_path="./logs_nlp/",
    eval_freq=1000,
    deterministic=True,
    render=False
)

# 모델 정의
#(1) 모델 정의
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log="./ppo_nlp_log/")

#(2) 학습 시작
model.learn(total_timesteps=10000, callback=eval_callback)

# 테스트
print("\n[모델 테스트]")
#(1) 질문 루프 시작
for prompt_id in [0, 1]:

    #(2) 질문 ID를 상태 벡터로 변환
    obs = np.array([prompt_id])

    #(2) 정책에 따라 행동 예측
    action, _ = model.predict(obs, deterministic=True)

    #(3) 정수 인덱스로 변환
    action = int(action)

    #(4) 질문 및 모델 응답 출력
    print(f"Q: {env.prompts[prompt_id]}")
    print(f"A: {env.responses[action]}")
    print("-" * 30)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to ./ppo_nlp_log/PPO_2
Eval num_timesteps=1000, episode_reward=0.80 +/- 0.40
Episode length: 1.00 +/- 0.00
---------------------------------
| eval/              |          |
|    mean_ep_length  | 1        |
|    mean_reward     | 0.8      |
| time/              |          |
|    total_timesteps | 1000     |
---------------------------------
New best mean reward!
Eval num_timesteps=2000, episode_reward=0.60 +/- 0.49
Episode length: 1.00 +/- 0.00
---------------------------------
| eval/              |          |
|    mean_ep_length  | 1        |
|    mean_reward     | 0.6      |
| time/              |          |
|    total_timesteps | 2000     |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 0.265    |
| time/              |          |
|    fps             | 1888     |
|    